In [7]:
from getEng import GetCredentials
from mysql.connector import connect
from getdata import getClient
from getdata import getEngine
import numpy as np

In [2]:
### match type of pandas df to hive type
def type_conversion(df, i):
    if df[i].dtypes == np.float64:
        column_type = "FLOAT"
    elif df[i].dtypes == np.int32:
        column_type = "INT"
    elif df[i].dtypes == object:
        column_type = "VARCHAR(255)"
    elif df[i].dtypes == np.dtype('datetime64[ns]'):
        column_type = "DATETIME"
    elif df[i].dtypes == np.int64:
        column_type = "INT"
    else:
        print(i)
        raise RuntimeError("No type")
    return column_type

In [3]:
### Table creation script is generated as string
def generate_create_statment(df, db_name, table_name):
    column_list = df.columns.values.tolist()
    create_statement = "CREATE TABLE " + db_name + "." + table_name + " ("
    for i in column_list:
        corrent_type = type_conversion(df, i)
        create_statement += i + " " + corrent_type + ", "
    create_statement = create_statement[:-2]
    create_statement = create_statement + ")"
    return create_statement

In [4]:
data = getClient()

query = ["SELECT * FROM `bigquery-public-data.new_york_taxi_trips.tlc_green_trips_2014` limit 5"]

df_sample = data.client.query(query[0]).result().to_dataframe()

In [5]:
df_sample.head()

,vendor_id,pickup_datetime,dropoff_datetime,store_and_fwd_flag,rate_code,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,...,mta_tax,tip_amount,tolls_amount,ehail_fee,total_amount,payment_type,distance_between_service,time_between_service,trip_type,imp_surcharge
0,2,2014-11-04 08:56:20,2014-11-04 09:42:00,N,1,-73.939430,40.805103,-73.977127,40.761135,1,...,0.5,4.0,0.0,NaN,31.5,1,NaN,NaN,1.0,NaN
1,1,2014-03-19 19:48:05,2014-03-19 20:18:17,N,1,-73.963966,40.807686,-73.986076,40.726524,1,...,0.5,0.0,0.0,NaN,31.0,2,NaN,NaN,NaN,NaN
2,1,2014-12-06 12:57:39,2014-12-06 13:31:10,N,1,-73.917618,40.743511,-73.837563,40.708107,1,...,0.5,6.9,0.0,NaN,41.4,1,NaN,NaN,1.0,NaN
3,1,2014-07-20 23:17:32,2014-07-21 00:02:43,N,1,-73.919327,40.758621,-73.859810,40.710022,1,...,0.5,0.0,0.0,NaN,45.0,2,NaN,NaN,1.0,NaN
4,2,2014-09-26 21:51:36,2014-09-26 22:30:13,N,1,-73.955475,40.690868,-73.822182,40.692451,1,...,0.5,0.0,0.0,NaN,32.0,2,NaN,NaN,1.0,NaN


In [20]:
db_name = 'taxi_data'
table_name = 'taxi_table_1'
statement_create = generate_create_statment(df_sample, db_name, table_name)
statement_create += " PARTITION BY HASH(Day(pickup_datetime))"

In [21]:
statement_create

'CREATE TABLE taxi_data.taxi_table_1 (vendor_id VARCHAR(255), pickup_datetime DATETIME, dropoff_datetime DATETIME, store_and_fwd_flag VARCHAR(255), rate_code INT, pickup_longitude FLOAT, pickup_latitude FLOAT, dropoff_longitude FLOAT, dropoff_latitude FLOAT, passenger_count INT, trip_distance FLOAT, fare_amount FLOAT, extra FLOAT, mta_tax FLOAT, tip_amount FLOAT, tolls_amount FLOAT, ehail_fee FLOAT, total_amount FLOAT, payment_type INT, distance_between_service FLOAT, time_between_service FLOAT, trip_type FLOAT, imp_surcharge FLOAT) PARTITION BY HASH(Day(pickup_datetime))'

In [22]:
cnx = connect(**vars(GetCredentials()))

In [23]:
cnx.cursor().execute(statement_create)